# Mutual Exclusion

__Critical Section Problem__: Piece of code, across all processes, for which we need to ensure there is at most one process executing it at any point of time.
    
Three functions involved
1. Enter
2. Access Resource: Operate upon exlusive resource
3. Exit

Very, very similar to semaphores in concurrent programming.
Acquire a lock before entering a protected section, release lock once done.
In fact, you might use these in a single host system.

Distributed system require something more clever.
You'll need to guarantee three properties
1. Safety - No more than one process in the Critical Section (CS) at once
2. Liveness - Every request for a CS is eventually granted
3. Ordering - Requests are granted sequentially. 
    * Not necessary, but desirable.
    


## Distributed Mutual Exclusion

Much more interesting than basic semaphore-based locking.

### System Model
* Reliable communications TCP
* Messages are delivered FIFO order
* Processes do not fail
    * Faut-tolerant variants exist in literature

### General Idea
* Elect a master
    * Master is the Keymaster
* Master keeps
    1. Queue of waiting requests for the Critical Section
    1. A special token (The Key allowing access to the Critical Section)
* Processes in the group may:
    * `enter`: Request access from master, wait for token
    * `exit`: Return token to master
* Master's actions:
    * On `enter`
        * If token available:
            * Give to process
        * Else:
            * Queue process
    * On `exit`
        * Pop head of queue
        * Give that process the token

Is it __safe__? Yes! One token goes around.
Is it __live__? Eventually! You'll get your token
* Unless someone with the token fails
* Or FIFO ordering is dis-respected.

Qualities of good mutual exlucstion algos:
1. Bandwidth
    * Minimal number of messages sent per `enter`/`exit`
    * Our example: 
        * 2 messages for enter
            * Process requests access
            * Master gives token
        * 1 message for exit
            * Process return token
1. Client delay
    * How long a process waits for `enter`/`exit` with no queue 
        * Our example: 2 (1 request + 1 grant)
1. Synchronization delay
    * How long a process waits for `enter`/`exit` with one person ahead in
    queue
        * Our example: 2 latencies (1 request + 1 grant)
    
**Master is the performance bottleneck**
**Master is a single-point of failure**

### Ring-based Mutual Exclusion
* Nodes can send messages to their successor
* *Token Message* transfers token to the successor
    * If successor doesn't want the token, it keeps passing.
* `enter`
    * Wait for token to reach you
* `exit`
    * Send token away
* Safety: 1 token
* Liveness: Eventually token traverses all nodes in the ring (again, system model disallows failures)
* Bandwidth
    * 1 message / `enter`; _N_ messages per node in system
    * 1 message per `exit`
* Client Delay
    * 0 to _N_ messages after waiting from `enter`
    * Best case: You already have the token! Good for you, no wait.
    * Worst case: Successor has the token, you have to wait for every node ahead of you.
* Synchronization Delay
    * 1 - _N_-1 messages
    * Best Case: `enter`ing process is successor of `exit`ing process
    * Worst Case: `enter`ing process is predecessor of `exit`ing process

## Ricart-Amgrawala's Algorithm

### System model
* Paired, reliable channels
* FIFO receipt of messages

Doesn't use a token
Leverages causality

### Algo
1. ``enter()`` broadcasted from process Pi
    * (_T_, Pi), _T_ = Pi's Lamport timestamp
1. Wait for all other processes to respond to request
    * Requests granted in order of causality
    * Pi used to break ties amongst equivalent timestamps
    
#### In Detail
* To enter critical section
    1. ``enter()`` f/ Pi
        * state = Wanted
        * Multicast (Ti, Pi) to all processes
        * Wait until receipt "Reply" f/ all processes
        * state = Held
        * Enter critical section
* On receipt of (Ti, Pi) at Pj
    1. if state is Held or Wanted && Tj < Ti
    * If we want or have the CS, and we have a lower timestamp than Pi
        1. Queue Pi's request
    1. Else, send "Reply"
* When exiting the CS
    1. state = Released
    1. Multicast "reply" to all queued requests

__Safety__
* Two processes can not access the CS at the same time
__Liveness__
* Wait for _N_-1 other processes
* Client/Synchronization delay down to O(1)
__Ordering__
* Lower timestamp goes first

But, bandwidth has gone up due to all the broadcasting

## Maekawa's Algorithm

Instead of requiring replies from all processes, accept only a subset of replies.

Each process Pi is associated with a __voting set__, Vi
Each belongs to its own voting set
The intersection of any two voting sets must be non-empty

Each voting set is of a constant size _K_
Each process belongs to _M_ other voting sets.
Maekawa recommends K = M = square_root(N)

### Algo
* Request CS
    * Request access from voting set
    * Wait for go-ahead from processes in voting set
* Done with CS
    * Tell voting set we're down with CS
* Asked for CS access
    * Queue the request if we have the CS or if we've already told someone else to enter
* Told the CS is free
    * Send the go-ahead to the next process in local queue 
    